# Bandstructures for 1D, 2D, and 3D nanowires
For more information see: B. Nijholt, A. R. Akhmerov, *Orbital effect of magnetic field on the Majorana phase diagram*, [arXiv:1509.02675](https://arxiv.org/abs/1509.02675) [[pdf](https://arxiv.org/pdf/1509.02675.pdf)], [Phys. Rev. B 93, 235434 (2016)](http://journals.aps.org/prb/abstract/10.1103/PhysRevB.93.235434).

In [ ]:
from copy import copy
from functools import partial
from operator import itemgetter
from time import time

import adaptive
import holoviews as hv
import kwant
import numpy as np
import pandas as pd
import sympy

import wires

adaptive.notebook_extension()
sympy.init_printing()

# hv.notebook_extension('bokeh')
hv.notebook_extension("matplotlib")

# Hamiltonian

In [ ]:
wires.get_sympy_hamiltonian({}, 2)

# System example

In [ ]:
%matplotlib inline
import matplotlib.pyplot

syst_pars = dict(a=10, L=100, dim=1)
syst = wires.make_wire(**syst_pars)
kwant.plot(syst)

In [ ]:
syst_pars = dict(a=10, L=100, dim=2, r=30)
syst = wires.make_wire(**syst_pars)
kwant.plot(syst)

In [ ]:
syst_pars = dict(a=10, L=100, dim=3, r=30)
syst = wires.make_wire(**syst_pars)
kwant.plot(syst)

# Band structures

In [ ]:
def bands(ylim, alpha, mu, Delta, B_x, B_y, B_z, params, lead_SC, lead_N):
    params = dict(params, alpha=alpha, mu=mu, Delta=Delta, B_x=B_x, B_y=B_y, B_z=B_z)
    ks = np.linspace(-0.9, 0.9, 201)
    Es = wires.bands(lead_N.finalized(), params, ks)
    kdims = [hv.Dimension(r"$k$", unit=r"nm$^{-1}$"), hv.Dimension(r"$E$", unit=r"meV")]
    normal = hv.Path((ks, Es), kdims=kdims, label="normal")
    Es = wires.bands(lead_SC.finalized(), params, ks)
    SC = hv.Path((ks, Es), kdims=kdims, label="SC")
    fermi_level = hv.HLine(0).opts(style=dict(linestyle="--"))
    return (normal * fermi_level).select(E=(-ylim, ylim)) + SC.select(E=(-ylim, ylim))


kdims = [
    hv.Dimension(("ylim"), range=(0.01, 50), default=5),
    hv.Dimension(("alpha", r"$\alpha$"), range=(0, 50), unit="meV·nm", default=20),
    hv.Dimension(("mu", r"$\mu$"), range=(0.0, 30), unit="meV", default=5),
    hv.Dimension(("Delta", r"$\Delta$"), range=(0, 1.0), unit="meV", default=0.25),
    hv.Dimension(("B_x", r"$B_x$"), range=(0, 10), unit="T", default=1),
    hv.Dimension(("B_y", r"$B_y$"), range=(0, 10), unit="T", default=0),
    hv.Dimension(("B_z", r"$B_z$"), range=(0, 0.5), unit="T", default=0),
]

## 1D band structure
$$H(x) = \left[ \frac{p_x^2}{2m} - \mu \right]\sigma_0\otimes\tau_z + \alpha p_x \sigma_y \otimes \tau_z + \frac{1}{2}B_x\sigma_x \otimes \tau_0  + \Delta\sigma_0\otimes\tau_x$$


In [ ]:
lead_SC = wires.make_lead_SC(10, dim=1)
lead_N = wires.make_lead_normal(10, with_holes=False, dim=1)

params = dict(V=0, g=50, **wires.constants)
bands_1D = partial(bands, params=params, lead_SC=lead_SC, lead_N=lead_N)
hv.DynamicMap(bands_1D, kdims=kdims)

## 2D band structure

$$H=\left(\frac{\mathbf{p}^{2}}{2m}-\mu\right)\sigma_{0}\otimes\tau_{z}+\alpha\left(p_{y}\sigma_{x}\otimes\tau_{z}-p_{x}\sigma_{y}\otimes\tau_{z}\right)+\frac{1}{2}B_{x}\sigma_{x}\otimes\tau_{0}+\frac{1}{2}B_{y}\sigma_{y}\otimes\tau_{0}+\frac{1}{2}B_{z}\sigma_{z}\otimes\tau_{0}+\Delta \sigma_{0}\otimes\tau_{x}$$

In [ ]:
syst_pars = dict(a=10, r=30, dim=2)
lead_SC = wires.make_lead_SC(**syst_pars)
lead_N = wires.make_lead_normal(**syst_pars, with_holes=False)

params = dict(a=syst_pars["a"], V=0, g=50, **wires.constants)
bands_2D = partial(bands, params=params, lead_SC=lead_SC, lead_N=lead_N)
hv.DynamicMap(bands_2D, kdims=kdims)

## 3D band structure


$$H=\left(\frac{\mathbf{p}^{2}}{2m}-\mu\right)\sigma_{0}\otimes\tau_{z}+\alpha\left(p_{y}\sigma_{x}\otimes\tau_{z}-p_{x}\sigma_{y}\otimes\tau_{z}\right)+\frac{1}{2}B_{x}\sigma_{x}\otimes\tau_{0}+\frac{1}{2}B_{y}\sigma_{y}\otimes\tau_{0}+\frac{1}{2}B_{z}\sigma_{z}\otimes\tau_{0}+\Delta \sigma_{0}\otimes\tau_{x}$$


In [ ]:
syst_pars = dict(a=10, r=30, dim=3)
lead_SC = wires.make_lead_SC(**syst_pars)
lead_N = wires.make_lead_normal(**syst_pars, with_holes=False)

params = dict(a=syst_pars["a"], V=0, g=50, **wires.constants)
bands_3D = partial(bands, params=params, lead_SC=lead_SC, lead_N=lead_N)
hv.DynamicMap(bands_3D, kdims=kdims)

# Wave functions 

## Wavefunction in the cross section of a 3D infinite lead.

In [ ]:
def wavefunctions(lead, momentum, p):
    h, t = lead.cell_hamiltonian(args=[p]), lead.inter_cell_hopping(args=[p])
    h_k = lambda k: h + t * np.exp(1j * k) + t.T.conj() * np.exp(-1j * k)
    vals, vecs = np.linalg.eigh(h_k(momentum))
    indxs = np.argsort(abs(vals))
    vecs = vecs[:, indxs]
    vals = vals[indxs]
    return vals, vecs


syst_pars = dict(a=10, r=30, dim=3)
lead_SC = wires.make_lead_SC(**syst_pars).finalized()

params = dict(
    a=syst_pars["a"],
    Delta=1,
    B_x=1,  # in units of meV
    mu=1,
    V=0,
    alpha=20,
    g=1,
    B_y=0,
    B_z=0,
    **wires.constants
)
params["mu_B"] = 2


wires.plot_wfs_in_cross_section(lead_SC, params, 0)

## Wavefunction of finite system

In [ ]:
syst_pars = dict(a=10, r=30, dim=1, L=4000)
syst = wires.make_wire(**syst_pars)

In [ ]:
params = dict(  # should be topological
    alpha=20, g=1, B_y=0, B_z=0, V=0, B_x=0.86, mu=0.42, Delta=0.384, **wires.constants
)
params["mu_B"] = 2

In [ ]:
H = syst.hamiltonian_submatrix(params=params)
Es, ψs = np.linalg.eigh(H)

indices = np.abs(Es).argsort()

rho = kwant.operator.Density(syst)
wf = rho(ψs[:, indices[0]])

hv.Curve(wf)